In [1]:
import pandas as pd
import datetime

In [2]:
# today's data and yesterday's date
today = (datetime.datetime.today().strftime('%d%b%Y')).upper()
yesterday = ((datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%d%b%Y')).upper()
today, yesterday

('21OCT2022', '20OCT2022')

In [3]:
# source url for fetching the data from nseIndia.com
source_url = 'https://archives.nseindia.com/content/historical/DERIVATIVES/2022/OCT/fo{}bhav.csv.zip'.format(today)  # CHANGE-ME
source_url

'https://archives.nseindia.com/content/historical/DERIVATIVES/2022/OCT/fo21OCT2022bhav.csv.zip'

In [4]:
# fetching the zip file
# fetching the csv file 
# NOTE : the zipfile must contain only csv file nothing else
temp_data = pd.read_csv(source_url,compression='zip')

In [5]:
# Dropping the unnecessary columns
temp_data = temp_data.drop(columns=['STRIKE_PR', 'OPEN', 'OPTION_TYP', 'HIGH', 'LOW', 'TIMESTAMP'])

In [6]:
# Dropping the last column as it was empty
temp_data.drop(temp_data.columns[[9]], axis=1, inplace=True)

In [7]:
# NIFTY 100 STOCK NAMES
companies = """ADANITRANS
HAL
BEL
SBIN
BANKBARODA
ADANIPORTS
SBICARD
EICHERMOT
SIEMENS
ITC
SBILIFE
NESTLEIND
ADANIENT
CHOLAFIN
BHARTIARTL
INDUSINDBK
INDIGO
HAVELLS
TATAMOTORS
DLF
M&M
APOLLOHOSP
LT
RELIANCE
NAUKRI
CIPLA
LTI
MUTHOOTFIN
HDFCLIFE
COALINDIA
HCLTECH
HINDALCO
TATAPOWER
ATGL
ICICIGI
TATACONSUM
ICICIBANK
BIOCON
ULTRACEMCO
TATASTEEL
WIPRO
HDFCAMC
IOC
MCDOWELL-N
ASIANPAINT
BAJAJHLDNG
PIIND
BERGEPAINT
INDUSTOWER
INFY
JSWSTEEL
MARUTI
SHREECEM
TCS
ONGC
ZOMATO
BOSCHLTD
TITAN
ICICIPRULI
BANDHANBNK
COLPAL
PIDILITIND
HINDUNILVR
UPL
POWERGRID
SRF
BAJFINANCE
MARICO
MPHASIS
IRCTC
BAJAJFINSV
ADANIGREEN
AXISBANK
HEROMOTOCO
DRREDDY
DABUR
GODREJCP
VEDL
PGHH
DIVISLAB
KOTAKBANK
GLAND
GAIL
LICI
BAJAJ-AUTO
BPCL
GRASIM
HDFCBANK
TECHM
DMART
BRITANNIA
SUNPHARMA
HDFC
NTPC
TORNTPHARM
NYKAA
PAYTM
AMBUJACEM
ACC
MOTHERSON
BANKNIFTY
NIFTY
"""
companies = companies.split('\n')
len(companies)

103

In [8]:
# Inplace droppping the row with symbol as FINNIFTY
# temp_data.drop(temp_data[temp_data['SYMBOL'] == 'FINNIFTY'].index, inplace=True)

# Filtering out data based on INSTRUMENT AND SYMBOL
temp_data = temp_data.loc[temp_data['INSTRUMENT'].isin(['FUTSTK', 'FUTIDX'])]
temp_data = temp_data.loc[temp_data['SYMBOL'].isin(companies)]

In [9]:
# Filtering out data based on current month expiry date
temp_data = temp_data[temp_data['EXPIRY_DT'].astype(str).str.match(r"\d{2}-Oct-2022")]

In [10]:
# Dropping out the first column i.e. INSTRUMENT
temp_data.drop(temp_data.columns[[0]], axis=1, inplace=True)

In [11]:
# new Column names (added date with the names)
column_names = [ 'SYMBOL', 'EXPIRY_DT', 'CLOSE'+today, 'SETTLE_PR'+today, 'CONTRACTS'+today, 'VAL_INLAKH'+today,'OPEN_INT'+today, 'CHG_IN_OI'+today]

# assigning the new column names
temp_data.columns = column_names

In [12]:
temp_data

,SYMBOL,EXPIRY_DT,CLOSE21OCT2022,SETTLE_PR21OCT2022,CONTRACTS21OCT2022,VAL_INLAKH21OCT2022,OPEN_INT21OCT2022,CHG_IN_OI21OCT2022
0,BANKNIFTY,27-Oct-2022,40788.30,40788.30,183408,1867031.68,1406925,-112600
16,NIFTY,27-Oct-2022,17564.45,17564.45,182444,1605093.51,10561900,-776500
34,ACC,27-Oct-2022,2260.15,2260.15,6153,34870.21,2398250,-820500
37,ADANIENT,27-Oct-2022,3312.15,3312.15,11100,184955.57,6383500,-2559000
40,ADANIPORTS,27-Oct-2022,800.75,800.75,11191,113175.64,22006250,-7185000
...,...,...,...,...,...,...,...,...
561,TORNTPHARM,27-Oct-2022,1578.15,1578.15,2028,16111.31,755000,-661000
576,ULTRACEMCO,27-Oct-2022,6369.85,6369.85,10940,69916.60,953900,-721100
579,UPL,27-Oct-2022,701.45,701.45,12522,115885.52,11996400,-5410600
582,VEDL,27-Oct-2022,280.15,280.15,10003,43501.26,14602550,-8704800


In [13]:
# writing the today's data to temp file to refresh the index.
temp_data.to_csv('temp_data.csv', index=False)

In [14]:
# reading the today's data from the temp file.
today_data = pd.read_csv('temp_data.csv')

# reading the main data from the mainfile
main_data = pd.read_csv('fandodata.csv')


In [15]:
# removing today's data's  expiry column as not required during merge
today_data.drop(columns='EXPIRY_DT', inplace=True)

In [16]:
# merging today's data with main data
main_data = main_data.merge(today_data, how='outer', on='SYMBOL')

In [17]:
# rearrange data
header_main = ['CLOSE'+yesterday, 'SETTLE_PR'+yesterday, 'CONTRACTS'+yesterday, 'VAL_INLAKH'+yesterday,'OPEN_INT'+yesterday, 'CHG_IN_OI'+yesterday]
header_new = ['CLOSE'+today, 'SETTLE_PR'+today, 'CONTRACTS'+today, 'VAL_INLAKH'+today,'OPEN_INT'+today, 'CHG_IN_OI'+today]

for i in range(6):
    new_head = header_new[i]
    prev_head = main_data.columns.get_loc(header_main[i]) + 1

    extracted_col = main_data[new_head]
    main_data.drop(columns=new_head, inplace=True)
    main_data.insert(prev_head, new_head, extracted_col)
    

In [18]:
main_data

,SYMBOL,EXPIRY_DT,CLOSE18OCT2022,CLOSE19OCT2022,CLOSE20OCT2022,CLOSE21OCT2022,SETTLE_PR18OCT2022,SETTLE_PR19OCT2022,SETTLE_PR20OCT2022,SETTLE_PR21OCT2022,...,VAL_INLAKH20OCT2022,VAL_INLAKH21OCT2022,OPEN_INT18OCT2022,OPEN_INT19OCT2022,OPEN_INT20OCT2022,OPEN_INT21OCT2022,CHG_IN_OI18OCT2022,CHG_IN_OI19OCT2022,CHG_IN_OI20OCT2022,CHG_IN_OI21OCT2022
0,BANKNIFTY,27-Oct-22,40380.10,40401.75,40205.35,40788.30,40380.10,40401.75,40205.35,40788.30,...,1605768.35,1867031.68,1633100,1587750,1519525,1406925,-194875,-45350,-68225,-112600
1,NIFTY,27-Oct-22,17502.00,17494.65,17540.70,17564.45,17502.00,17494.65,17540.70,17564.45,...,1655734.25,1605093.51,11411250,10987150,11338400,10561900,-1273300,-424100,351250,-776500
2,ACC,27-Oct-22,2224.85,2247.75,2286.10,2260.15,2224.85,2247.75,2286.10,2260.15,...,34795.23,34870.21,4131250,3697000,3218750,2398250,-530500,-434250,-478250,-820500
3,ADANIENT,27-Oct-22,3291.45,3275.35,3358.10,3312.15,3291.45,3275.35,3358.10,3312.15,...,240114.19,184955.57,13333500,12281000,8942500,6383500,-1091000,-1052500,-3338500,-2559000
4,ADANIPORTS,27-Oct-22,810.00,805.25,819.70,800.75,810.00,805.25,819.70,800.75,...,89468.41,113175.64,35147500,33065000,29191250,22006250,-34007500,-2082500,-3873750,-7185000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,TORNTPHARM,27-Oct-22,1564.55,1580.40,1593.60,1578.15,1564.55,1580.40,1593.60,1578.15,...,9146.05,16111.31,1736500,1662000,1416000,755000,18500,-74500,-246000,-661000
87,ULTRACEMCO,27-Oct-22,6368.65,6415.05,6317.60,6369.85,6368.65,6415.05,6317.60,6369.85,...,61010.07,69916.60,1871700,2010000,1675000,953900,-40000,138300,-335000,-721100
88,UPL,27-Oct-22,683.75,678.30,712.85,701.45,683.75,678.30,712.85,701.45,...,125267.37,115885.52,23280400,22100000,17407000,11996400,-465400,-1180400,-4693000,-5410600
89,VEDL,27-Oct-22,280.60,277.30,283.50,280.15,280.60,277.30,283.50,280.15,...,35040.25,43501.26,29367850,28287500,23307350,14602550,-94550,-1080350,-4980150,-8704800


In [19]:
main_data.to_csv('fandodata.csv', index=False)